# Image Captioning using LSTM on Flickr8K dataset

##1. Downloading Data and Dependencies

In [ ]:
#downloading the data
#!wget https://drive.google.com/file/d/1zF82SKqkmKGD-WYF0P5A9rglcTqwLFPz/view?usp=sharing
#!wget https://drive.google.com/file/d/10aFztxdXii4lKHP3-aaUCpR2nR5BkOW4/view?usp=sharing

In [ ]:
!pip install --upgrade nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [ ]:
#importing the libraries
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from tqdm import tqdm
from keras.applications import vgg16
from keras.utils import load_img,img_to_array
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score

from numpy import array
import pickle
import numpy as np
import os
import cv2

##2. Data Visualization and Preprocessing

In [ ]:
#converting the text files to pandas dataframe
image_tokens=pd.read_csv("/kaggle/input/vr-part-2-flickr8k/Flickr8k_text/Flickr8k_text/Flickr8k.lemma.token.txt",sep='\t',names=["img_id","img_caption"])
train_image_names=pd.read_csv("/kaggle/input/vr-part-2-flickr8k/Flickr8k_text/Flickr8k_text/Flickr_8k.trainImages.txt",names=["img_id"])
test_image_names=pd.read_csv("/kaggle/input/vr-part-2-flickr8k/Flickr8k_text/Flickr8k_text/Flickr_8k.testImages.txt",names=["img_id"])
val_image_names=pd.read_csv("/kaggle/input/vr-part-2-flickr8k/Flickr8k_text/Flickr8k_text/Flickr_8k.valImages.txt",names=["img_id"])

In [ ]:
#just checking the number of images in the dataset
l=os.listdir("/kaggle/input/vr-part-2-flickr8k/Flicker8k_Images/Flicker8k_Images")
print(len(l))

8091


In [ ]:
#visualising one of the train images
img=cv2.imread("/kaggle/input/vr-part-2-flickr8k/Flicker8k_Images/Flicker8k_Images"+ "/" +train_image_names.img_id[0])
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
#removing the #0,#1,#2,#3,#5 from the image ids
image_tokens["img_id"]=image_tokens["img_id"].map(lambda x: x[:len(x)-2])

In [ ]:
image_tokens["img_caption"]=image_tokens["img_caption"].map(lambda x: "<start> " + x.strip() + " <end>")

In [ ]:
#head of the image_tokens dataframe
image_tokens.head()

,img_id,img_caption
0,1305564994_00513f9a5b.jpg,<start> A man in street racer armor be examine...
1,1305564994_00513f9a5b.jpg,<start> Two racer drive a white bike down a ro...
2,1305564994_00513f9a5b.jpg,<start> Two motorist be ride along on their ve...
3,1305564994_00513f9a5b.jpg,<start> Two person be in a small race car driv...
4,1305564994_00513f9a5b.jpg,<start> Two person in race uniform in a street...


In [ ]:
#head of the train_image_names dataframe
train_image_names.head()

,img_id
0,2513260012_03d33305cf.jpg
1,2903617548_d3e38d7f88.jpg
2,3338291921_fe7ae0c8f8.jpg
3,488416045_1c6d903fe0.jpg
4,2644326817_8f45080b87.jpg


### Creating dictionaries to map image_id and their corresponding captions

In [ ]:
#creating train dictionary having key as the image id and value as a list of its captions
train_captions={}
for i in tqdm(range(len(train_image_names))):
  l=[caption for caption in(image_tokens[image_tokens["img_id"]==train_image_names["img_id"].iloc[i]].img_caption)]
  train_captions[train_image_names["img_id"].iloc[i]]=l

100%|██████████| 6000/6000 [00:35<00:00, 167.46it/s]


In [ ]:
#creating test dictionary having key as the image id and value as a list of its captions
test_captions={}
for i in tqdm(range(len(test_image_names))):
  l=[caption for caption in(image_tokens[image_tokens["img_id"]==test_image_names["img_id"].iloc[i]].img_caption)]
  test_captions[test_image_names["img_id"].iloc[i]]=l

100%|██████████| 1000/1000 [00:06<00:00, 161.22it/s]


In [ ]:
#creating validation dictionary having key as the image id and value as a list of its captions
validation_captions={}
for i in tqdm(range(len(val_image_names))):
  l=[caption for caption in(image_tokens[image_tokens["img_id"]==val_image_names["img_id"].iloc[i]].img_caption)]
  validation_captions[val_image_names["img_id"].iloc[i]]=l

100%|██████████| 1000/1000 [00:05<00:00, 168.32it/s]


### VGG model for encoding images

In [ ]:
model=VGG19(include_top=False, weights='imagenet',pooling='avg',input_shape=(224,224,3))
model.summary()

80134624/80134624 [==============================] - 4s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

### Encoding images and forming dictionaries containing mapping of image_id to image encodings

In [ ]:
#extracting image encodings(features) from resnet50 and forming dict train_features
path="/kaggle/input/vr-part-2-flickr8k/Flicker8k_Images/Flicker8k_Images/"
train_features={}
c=0
for image_name in tqdm(train_captions):
  img_path=path+image_name
  img=load_img(img_path,target_size=(224,224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  train_features[image_name]=features.squeeze()

  0%|          | 0/6000 [00:00<?, ?it/s]

1/1 [==============================] - 5s 5s/step


  0%|          | 1/6000 [00:04<8:09:36,  4.90s/it]

1/1 [==============================] - 0s 19ms/step


  0%|          | 3/6000 [00:05<2:12:08,  1.32s/it]

...

100%|█████████▉| 5996/6000 [07:16<00:00, 13.01it/s]

1/1 [==============================] - 0s 18ms/step


100%|█████████▉| 5998/6000 [07:17<00:00, 13.27it/s]

1/1 [==============================] - 0s 18ms/step


100%|██████████| 6000/6000 [07:17<00:00, 13.72it/s]


In [ ]:
with open( "train_encoded_images.p", "wb" ) as pickle_f:
    pickle.dump(train_features, pickle_f )

In [ ]:
#extracting image encodings(features) from resnet50 and forming dict test_features
path="/kaggle/input/vr-part-2-flickr8k/Flicker8k_Images/Flicker8k_Images/"
test_features={}
c=0
for image_name in tqdm(test_captions):
  img_path=path+image_name
  img=load_img(img_path,target_size=(224,224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  test_features[image_name]=features.squeeze()

  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step


  0%|          | 2/1000 [00:00<01:17, 12.82it/s]

1/1 [==============================] - 0s 20ms/step


  0%|          | 4/1000 [00:00<01:12, 13.82it/s]

...

100%|█████████▉| 996/1000 [01:15<00:00, 13.59it/s]

1/1 [==============================] - 0s 18ms/step


100%|█████████▉| 998/1000 [01:15<00:00, 13.66it/s]

1/1 [==============================] - 0s 18ms/step


100%|██████████| 1000/1000 [01:15<00:00, 13.28it/s]


In [ ]:
with open( "test_encoded_images.p", "wb" ) as pickle_f:
    pickle.dump(test_features, pickle_f )

In [ ]:
#extracting image encodings(features) from resnet50 and forming dict validation_features
path="/kaggle/input/vr-part-2-flickr8k/Flicker8k_Images/Flicker8k_Images/"
validation_features={}
c=0
for image_name in tqdm(validation_captions):
  img_path=path+image_name
  img=load_img(img_path,target_size=(224,224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  validation_features[image_name]=features.squeeze()

  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 18ms/step


  0%|          | 2/1000 [00:00<01:07, 14.70it/s]

1/1 [==============================] - 0s 21ms/step


  0%|          | 4/1000 [00:00<01:09, 14.38it/s]

...

100%|█████████▉| 996/1000 [01:13<00:00, 14.28it/s]

1/1 [==============================] - 0s 18ms/step


100%|█████████▉| 998/1000 [01:13<00:00, 14.72it/s]

1/1 [==============================] - 0s 18ms/step


100%|██████████| 1000/1000 [01:13<00:00, 13.53it/s]


In [ ]:
with open( "validation_encoded_images.p", "wb" ) as pickle_f:
    pickle.dump(validation_features, pickle_f )

### Setting hyper parameters for vocabulary size and maximum length

In [ ]:
all_captions=[]
for img_id in tqdm(train_captions):
  for captions in train_captions[img_id]:
    all_captions.append(captions)

all_words=" ".join(all_captions)
print()
print(len(all_words))
unique_words=list(set(all_words.strip().split(" ")))
print(len(unique_words))

100%|██████████| 6000/6000 [00:00<00:00, 421185.34it/s]


1991192


6328


In [ ]:
#defining max_length and vocabulary size
vocab_size=len(unique_words)+1
max_length=40

### Creating dictionaries containg mapping of words to indices and indices to words

In [ ]:
#forming dictionaries containg mapping of words to indices and indices to words
words_to_indices={val:index+1 for index, val in enumerate(unique_words)}
indices_to_words = { index+1:val for index, val in enumerate(unique_words)}
words_to_indices["Unk"]=0
indices_to_words[0]="Unk"

### Transforming data into dictonary mapping of image_id to encoded captions

In [ ]:
#forming dictionary having encoded captions
train_encoded_captions={}
for img_id in tqdm(train_captions):
  train_encoded_captions[img_id]=[]
  for i in range(5):
    train_encoded_captions[img_id].append([words_to_indices[s] for s in train_captions[img_id][i].split(" ")])

100%|██████████| 6000/6000 [00:00<00:00, 48662.34it/s]


In [ ]:
for img_id in tqdm(train_encoded_captions):
  print(train_encoded_captions[img_id])
  break

  0%|          | 0/6000 [00:00<?, ?it/s]

[[1303, 5227, 4240, 6129, 5687, 2815, 1613, 4678, 323, 6129, 1579, 4763, 5194, 5537, 130], [1303, 2586, 6129, 2522, 52, 6129, 2799, 5194, 130], [1303, 634, 6129, 2522, 4077, 4980, 3793, 4763, 5752, 1879, 5537, 130], [1303, 634, 6129, 1435, 272, 1579, 4763, 5194, 5537, 130], [1303, 634, 6129, 2815, 2799, 4678, 3636, 4816, 4109, 2452, 4882, 5537, 130]]


In [ ]:
for img_id in tqdm(train_encoded_captions):
  train_encoded_captions[img_id]=pad_sequences(train_encoded_captions[img_id], maxlen=max_length, padding='post')

100%|██████████| 6000/6000 [00:00<00:00, 25964.49it/s]


In [ ]:
for img_id in tqdm(train_encoded_captions):
  print(train_encoded_captions[img_id])
  break

  0%|          | 0/6000 [00:00<?, ?it/s]

[[1303 5227 4240 6129 5687 2815 1613 4678  323 6129 1579 4763 5194 5537
   130    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [1303 2586 6129 2522   52 6129 2799 5194  130    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [1303  634 6129 2522 4077 4980 3793 4763 5752 1879 5537  130    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [1303  634 6129 1435  272 1579 4763 5194 5537  130    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [1303  634 6129 2815 2799 4678 3636 4816 4109 2452 4882 5537  130    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  

##3. Playing with different embeddings :

### Word embeddings: 
a. BERT \
b. Word2vec \
c. Glove

In [ ]:
import gensim
import torch
# from transformers import BertModel, BertTokenizer

# Load pre-trained Word2Vec model
# w2v_model = gensim.models.KeyedVectors.load_word2vec_format('path/to/pretrained/word2vec/model')



# Define a function to encode the captions using Word2Vec embeddings
#def encode_captions_w2v(captions, w2v_model):
#    encoded_captions = []
#    for caption in captions:
#        encoded_caption = []
#        for word in caption.split():
#            if word in w2v_model:
#                encoded_caption.append(w2v_model[word])
#        encoded_captions.append(torch.tensor(encoded_caption))
#    return encoded_captions

# Define a function to encode the captions using BERT embeddings
def encode_captions_bert(captions, bert_model, bert_tokenizer):
    encoded_captions = []
    for caption in captions:
        encoded_caption = bert_tokenizer.encode(caption, add_special_tokens=True, return_tensors='pt')
        encoded_captions.append(bert_model(encoded_caption)[0])
    return encoded_captions



# Encode the captions using BERT embeddings (Uncomment this to use BERT Embeddings)
# Load pre-trained BERT model and tokenizer
# bert_model = BertModel.from_pretrained('bert-base-uncased')
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#train_encoded_captions = encode_captions_bert(train_encoded_captions, bert_model, bert_tokenizer)
#test_encoded_captions = encode_captions_bert(test_encoded_captions, bert_model, bert_tokenizer)
#val_encoded_captions = encode_captions_bert(val_encoded_captions, bert_model, bert_tokenizer)


##4. Data Generator for Modelling

In [ ]:
def data_generator(train_encoded_captions,train_features,num_of_photos):
  X1, X2, Y = list(), list(), list()
  max_length=40
  n=0
  for img_id in tqdm(train_encoded_captions):
    n+=1
    for i in range(5):
      for j in range(1,40):
        curr_sequence=train_encoded_captions[img_id][i][0:j].tolist()
        next_word=train_encoded_captions[img_id][i][j]
        curr_sequence=pad_sequences([curr_sequence], maxlen=max_length, padding='post')[0]
        one_hot_next_word=to_categorical([next_word],vocab_size)[0]
        X1.append(train_features[img_id])
        X2.append(curr_sequence)
        Y.append(one_hot_next_word)
    if(n==num_of_photos):
      yield [[array(X1), array(X2)], array(Y)]
      X1, X2, Y = list(), list(), list()
      n=0

##5. Modelling

In [ ]:
#importing dependencies
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import add
from keras.layers import Embedding

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
for i in train_features:
  print(train_features[i].shape)
  break

(512,)


##6. Model

In [ ]:
#model
input_1=Input(shape=(2048,))
dropout_1=Dropout(0.2)(input_1)
dense_1=Dense(256,activation='relu')(dropout_1)

input_2=Input(shape=(max_length,))
embedding_1=Embedding(vocab_size,256)(input_2)
dropout_2=Dropout(0.2)(embedding_1)
lstm_1=LSTM(256)(dropout_2)

add_1=add([dense_1,lstm_1])
dense_2=Dense(256,activation='relu')(add_1)
dense_3=Dense(vocab_size,activation='softmax')(dense_2)

model=Model(inputs=[input_1,input_2],outputs=dense_3)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


##7. Training

In [ ]:
epochs=1
no_of_photos=5
steps=len(train_encoded_captions)//no_of_photos
for i in range(epochs):
  generator=data_generator(train_encoded_captions,train_features,no_of_photos)
  model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)

In [ ]:
epochs=5
no_of_photos=5
steps=len(train_encoded_captions)//no_of_photos
for i in range(epochs):
  generator=data_generator(train_encoded_captions,train_features,no_of_photos)
  model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)

In [ ]:
test_features

##8. Greedy Search function

In [ ]:
def greedy_search(photo):
  photo=photo.reshape(1,2048)
  in_text='<start>'
  for i in range(max_length):
    sequence = [words_to_indices[s] for s in in_text.split(" ") if s in words_to_indices]
    sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
    y_pred = model.predict([photo,sequence],verbose=0)
    y_pred = np.argmax(y_pred[0])
    word = indices_to_words[y_pred]
    in_text += ' ' + word
    if word == '<end>':
      break
  final = in_text.split()
  final = final[1:-1]
  #final = " ".join(final)
  return final

##9. Predicting Captions on Test Set using Greedy Search

In [ ]:
i=0
for img_id in test_features:
  i+=1
  img=cv2.imread("/kaggle/input/vr-part-2-flickr8k/Flicker8k_Images/Flicker8k_Images"+ "/" + img_id)
  plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
  photo=test_features[img_id]
  plt.show()
  reference=[]
  for caps in test_captions[img_id]:
    list_caps=caps.split(" ")
    list_caps=list_caps[1:-1]
    reference.append(list_caps)
  candidate=greedy_search(photo)
  score = sentence_bleu(reference, candidate)
  print("Referance Captions: ")
  for cap in reference:
    print(" ".join(cap))
  print("Predicted Caption: ")
  print(" ".join(candidate))
  print("bleu score: ",score)
  if(i==5):
    break

##10. Calculating Average Bleu Score on Test Set using Greedy Search

In [ ]:
i=0
tot_score=0
for img_id in tqdm(test_features):
  i+=1
  photo=test_features[img_id]
  reference=[]
  for caps in test_captions[img_id]:
    list_caps=caps.split(" ")
    list_caps=list_caps[1:-1]
    reference.append(list_caps)
  candidate=greedy_search(photo)
  score = sentence_bleu(reference, candidate)
  tot_score+=score
avg_score=tot_score/i
print()
print("Bleu score on Greedy search")
print("Score: ",avg_score)


Bleu score on Greedy search
Score:  0.35


##11. Calculate Average METEOR Score on test set using Greedy Search

In [ ]:
i=0
tot_score=0
for img_id in tqdm(test_features):
  i+=1
  photo=test_features[img_id]
  reference=[]
  for caps in test_captions[img_id]:
    list_caps=caps.split(" ")
    list_caps=list_caps[1:-1]
    reference.append(list_caps)
  candidate=greedy_search(photo)
  score = meteor_score(reference, candidate)
  tot_score+=score
avg_score=tot_score/i
print()
print("METEOR score on Greedy search")
print("Score: ",avg_score)


METEOR score on Greedy search
Score:  0.22


##12. Beam Search Function

In [ ]:
def beam_search(photo,k):
  photo=photo.reshape(1,2048)
  in_text='<start>'
  sequence = [words_to_indices[s] for s in in_text.split(" ") if s in words_to_indices]
  sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
  y_pred = model.predict([photo,sequence],verbose=0)
  predicted=[]
  y_pred=y_pred.reshape(-1)
  for i in range(y_pred.shape[0]):
    predicted.append((i,y_pred[i]))
  predicted=sorted(predicted,key=lambda x:x[1])[::-1]
  b_search=[]
  for i in range(k):
    word = indices_to_words[predicted[i][0]]
    b_search.append((in_text +' ' + word,predicted[i][1]))
    
  for idx in range(max_length):
    b_search_square=[]
    for text in b_search:
      if text[0].split(" ")[-1]=="<end>":
        break
      sequence = [words_to_indices[s] for s in text[0].split(" ") if s in words_to_indices]
      sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
      y_pred = model.predict([photo,sequence],verbose=0)
      predicted=[]
      y_pred=y_pred.reshape(-1)
      for i in range(y_pred.shape[0]):
        predicted.append((i,y_pred[i]))
      predicted=sorted(predicted,key=lambda x:x[1])[::-1]
      for i in range(k):
        word = indices_to_words[predicted[i][0]]
        b_search_square.append((text[0] +' ' + word,predicted[i][1]*text[1]))
    if(len(b_search_square)>0):
      b_search=(sorted(b_search_square,key=lambda x:x[1])[::-1])[:5]
  final=b_search[0][0].split()
  final = final[1:-1]
  #final=" ".join(final)
  return final

##13. Calculating Average Bleu Score on Test Set using Beam Search with k=3

In [ ]:
i=0
tot_score=0
for img_id in tqdm(test_features):
  i+=1
  photo=test_features[img_id]
  reference=[]
  for caps in test_captions[img_id]:
    list_caps=caps.split(" ")
    list_caps=list_caps[1:-1]
    reference.append(list_caps)
  candidate=beam_search(photo,3)
  score = sentence_bleu(reference, candidate)
  tot_score+=score
avg_score=tot_score/i
print()
print("Bleu score on Beam search with k=3")
print("Score: ",avg_score)


Bleu score on Beam search with k=3
Score:  0.45


##14. Calculate Average METEOR Score on test set using Beam Search k=3

In [ ]:
i=0
tot_score=0
for img_id in tqdm(test_features):
  i+=1
  photo=test_features[img_id]
  reference=[]
  for caps in test_captions[img_id]:
    list_caps=caps.split(" ")
    list_caps=list_caps[1:-1]
    reference.append(list_caps)
  candidate=beam_search(photo,3)
  score = meteor_score(reference, candidate)
  tot_score+=score
avg_score=tot_score/i
print()
print("METEOR score on Beam search with k=3")
print("Score: ",avg_score)

METEOR score on Beam search with k=3
Score:  0.28
